##Making the Model

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import zipfile 
import os

from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from google.colab import drive

You need to have your Kaggle API token to download this dataset from kaggle. The name of the file should be 'kaggle.json'. You can download it from your account's settings. You can manually download the dataset and upload it to google colab https://www.kaggle.com/muhammadghazy/weightheightbmicategoryv2 .

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"muhammadghazy","key":"46ce4c67d3e08a2733a683cadd60815e"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d muhammadghazy/weightheightbmicategoryv2

  0% 0.00/911k [00:00<?, ?B/s]
100% 911k/911k [00:00<00:00, 59.5MB/s]


In [ ]:
zip_ref = zipfile.ZipFile("weightheightbmicategoryv2.zip", "r")
zip_ref.extractall("files")
zip_ref.close()

In [ ]:
!ls

files  kaggle.json  sample_data  weightheightbmicategoryv2.zip


In [ ]:
data_train = pd.read_excel('/content/files/weight-height-bmi-v2.xlsx')
data_train_label = pd.read_excel('/content/files/weight-height-bmi-v2.xlsx').pop("category")
data_train_label

0             Obese
1       Underweight
2        Overweight
3        Overweight
4        Overweight
           ...     
9995    Underweight
9996     Overweight
9997    Underweight
9998     Overweight
9999    Underweight
Name: category, Length: 10000, dtype: object

In [ ]:
data_train.drop("bmi",inplace=True,axis=1)
data_train.drop("id",inplace=True,axis=1)
data_train.drop("category",inplace=True,axis=1)
data_train_features = np.array(data_train)

In [ ]:
def conv(x):
  if x=="Underweight":
    return 0
  if x=="Normal":
    return 1
  if x=="Overweight":
    return 2
  if x=="Obese":
    return 3

data_train_label = data_train_label.map(conv)

In [ ]:
model = tf.keras.Sequential([
                             layers.Dense(2),
                             layers.Dense(32,activation=tf.nn.relu),
                             layers.Dense(32,activation=tf.nn.relu),
                             layers.Dense(4,activation="softmax")
])


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(data_train_features, data_train_label, epochs=40)

Epoch 1/40
313/313 [==============================] - 1s 1ms/step - loss: 2.6543 - accuracy: 0.6528
Epoch 2/40
313/313 [==============================] - 0s 1ms/step - loss: 0.5364 - accuracy: 0.7589
Epoch 3/40
313/313 [==============================] - 0s 1ms/step - loss: 0.5422 - accuracy: 0.7580
Epoch 4/40
313/313 [==============================] - 0s 1ms/step - loss: 0.5536 - accuracy: 0.7591
Epoch 5/40
313/313 [==============================] - 0s 1ms/step - loss: 0.5227 - accuracy: 0.7699
Epoch 6/40
313/313 [==============================] - 0s 1ms/step - loss: 0.5318 - accuracy: 0.7588
Epoch 7/40
313/313 [==============================] - 0s 1ms/step - loss: 0.5085 - accuracy: 0.7727
Epoch 8/40
313/313 [==============================] - 0s 1ms/step - loss: 0.5323 - accuracy: 0.7641
Epoch 9/40
313/313 [==============================] - 0s 1ms/step - loss: 0.5110 - accuracy: 0.7732
Epoch 10/40
313/313 [==============================] - 0s 1ms/step - loss: 0.4916 - accuracy: 0.7802

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 2)                 6         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                96        
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 132       
Total params: 1,290
Trainable params: 1,290
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def matching(array,classes):
  i = array[0].tolist().index(np.max(array))
  return classes[i]

category = ["Underweight","Normal","Overweight","Obese"]

Testing the Model

In [ ]:
berat = float(input("Berapa berat kamu?(Kg) "))
tinggi = float(input("Berapa tinggi kamu?(Cm) "))
  
data = np.array([[tinggi,berat]])
pred = model.predict(data)
result = matching(pred,category)
print("Dengan berat {} kg dan tinggi {} m, maka kondisi badan kamu adalah {}".format(berat,tinggi,result))

Berapa berat kamu?(Kg) 40
Berapa tinggi kamu?(Cm) 175
Dengan berat 40.0 kg dan tinggi 175.0 m, maka kondisi badan kamu adalah Underweight


## Export Model to .tflite

In [ ]:
# EXERCISE: Use the tf.saved_model API to save your model in the SavedModel format. 
export_dir = '/media/1'
tf.saved_model.save(model,export_dir)

INFO:tensorflow:Assets written to: /media/1/assets


In [ ]:
# Select mode of optimization
mode = "Speed" 

if mode == 'Storage':
    optimization = tf.lite.Optimize.OPTIMIZE_FOR_SIZE
elif mode == 'Speed':
    optimization = tf.lite.Optimize.OPTIMIZE_FOR_LATENCY
else:
    optimization = tf.lite.Optimize.DEFAULT

In [ ]:
# EXERCISE: Use the TFLiteConverter SavedModel API to initialize the converter
import pathlib

converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)

# Set the optimzations
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_LATENCY]# YOUR CODE HERE

# Invoke the converter to finally generate the TFLite model
tflite_model = converter.convert()# YOUR CODE HERE

In [ ]:
tflite_model_file = pathlib.Path('./model.tflite')
tflite_model_file.write_bytes(tflite_model)

4288

##Deploying to firebase

In [ ]:
import firebase_admin
from firebase_admin import ml
from firebase_admin import credentials


firebase_admin.initialize_app(
  credentials.Certificate("serviceAccountKey.json"),options={
      'storageBucket': 'gs://ninth-incentive-312907.appspot.com',
  })

In [ ]:
# First, import and initialize the SDK as shown above.

# Load a tflite file and upload it to Cloud Storage
source = ml.TFLiteGCSModelSource.from_tflite_model_file('model.tflite')

# Create the model object
tflite_format = ml.TFLiteFormat(model_source=source)
model = ml.Model(
    display_name="BMI_Predictor",  # This is the name you use from your app to load the model.
    tags=["ML_model"],             # Optional tags for easier management.
    model_format=tflite_format)

# Add the model to your Firebase project and publish it
new_model = ml.create_model(model)
ml.publish_model(new_model.model_id)